In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
import annoy
import pandas as pd

%load_ext autoreload
%autoreload 2
from ann_utils import ANN

In [ ]:
# softmax models
#model = "2021-02-20_16:57:56.170925" # medium classifier
#model = "2021-02-21_13:51:24.722105" # no classifier
#model = "2021-02-22_10:44:33.178333" # small classifier

# sigmoid models
#model = "2021-02-23_18:31:57.627258" # no classifier
#model = "2021-02-24_08:33:26.520580" # small classifier
#model = "2021-02-23_07:54:35.951167" # medium classifier
#model = "2021-02-25_07:33:41.814662" # no decoder

In [ ]:
embedding_path = f"models/{model}/fma_embeddings.npy"
label_path = f"models/{model}/fma_labels.npy"
filename_path = "data/fma_small_wav/filenames.npy"

In [ ]:
ann = ANN(embedding_path, label_path, filename_path, -1)
ann.supervised_evaluation(double_vote=False)

In [ ]:
# this takes a very long time to compute (even longer than the evaluation)
#ann.plot_tsne(size=1000) # how many random points you want to fit_transform 

In [ ]:
c_mat = ann.confusion_matrix

fig, ax = plt.subplots(figsize=(8, 6), dpi=160)
ax = sns.heatmap(c_mat/np.sum(c_mat, axis=1)[:,np.newaxis], ax = ax,
                 fmt='.1%', annot=True, cmap='Blues', norm=LogNorm(1e-2,1),
                xticklabels=ann.genre_names, yticklabels=ann.genre_names)

ax.set_ylabel("Labels")
ax.set_xlabel("Predictions")
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax.set_title("Confusion Matrix (in \%)");

In [ ]:
report = ann.classification_report
df = pd.DataFrame.from_dict(report).transpose()
df.head(df.shape[0]-3)

In [ ]:
df.tail(3)